https://towardsdatascience.com/multivariate-linear-regression-in-python-step-by-step-128c2b127171

https://www.mrlc.gov/data/legends/national-land-cover-database-2016-nlcd2016-legend

https://medium.com/analytics-vidhya/implementing-linear-regression-using-sklearn-76264a3c073c


In [3]:
# !pip install scikit-learn

In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.linear_model import LinearRegression
pd.set_option('display.max_columns', None)

In [2]:
catch_file = './Catchment Data/Catchment_Regression_Data.csv'
catch_data = pd.read_csv(catch_file)
catch_data

,Unnamed: 0,CatchId,SLOPE,LENGTHKM,AREASQKM,StreamOrde,Avg_Lon,Avg_Lat,Avg_Elev,RunoffC,MajLC,Roughness
0,0,932020122,0.049157,4.754,6.0120,1.0,-75.185438,43.127256,205.3807,NaN,NaN,0.050000
1,1,3246072,0.004730,2.260,1.6632,4.0,-74.565526,42.681145,371.1816,45.21915,81.0,0.005000
2,2,3246388,0.000030,0.168,0.0378,3.0,-74.156619,42.612042,285.9409,43.21429,81.0,0.005000
3,3,3247620,0.005577,0.624,1.0278,1.0,-74.116866,42.232266,716.3624,45.54291,43.0,0.142483
4,4,3247570,0.009604,21.108,80.7237,1.0,-74.214211,42.256132,650.3924,41.60782,41.0,0.228417
...,...,...,...,...,...,...,...,...,...,...,...,...
43279,43279,10466733,0.000030,0.336,0.1845,NaN,NaN,NaN,NaN,NaN,NaN,0.050000
43280,43280,10466483,0.000030,0.378,0.3438,NaN,NaN,NaN,NaN,NaN,NaN,0.050000
43281,43281,10469587,0.002440,1.168,0.9054,NaN,NaN,NaN,NaN,NaN,NaN,0.050000
43282,43282,10469373,0.000551,0.363,0.3555,NaN,NaN,NaN,NaN,NaN,NaN,0.050000


In [3]:
# ADD Corrections determined in Notebook 01.2
catch_data.loc[catch_data.CatchId == 6251308, 'SLOPE'] = 0.15 / 5223 # PASS1
catch_data.loc[catch_data.CatchId == 6251234, 'SLOPE'] = 0.36 / 6934 # PASS2 
catch_data.loc[catch_data.CatchId == 6251128, 'SLOPE'] = 1.27 / 1729 # PASS3
catch_data.loc[catch_data.CatchId == 6249784, 'SLOPE'] = 0.00005 # PASS7 - arbitrary because slope is almost zero
catch_data.loc[catch_data.CatchId == 6251122, 'SLOPE'] = 0.37 / (694 + 701) # PASS10
catch_data.loc[catch_data.CatchId == 6251144, 'SLOPE'] = 0.37 / (694 + 701) # PASS10
catch_data.loc[catch_data.CatchId == 6250358, 'SLOPE'] = 0.16 / 2744 # PASS16
catch_data.loc[catch_data.CatchId == 6250388, 'SLOPE'] = 0.71 / 3277 # PASS17
catch_data.loc[catch_data.CatchId == 6250418, 'SLOPE'] = 1.40 / 2824 # PASS18

catch_data.loc[catch_data.CatchId == 6246206, 'SLOPE'] = 1.59 / 2824 # WANQ2
catch_data.loc[catch_data.CatchId == 6250622, 'SLOPE'] = 1.41 / 2480 # POMP4/5
catch_data.loc[catch_data.CatchId == 6249738, 'SLOPE'] = 0.00005     # POMP2 - arbitrary value to remove error
# catch_data[catch_data.CatchId == 6246098, 'SLOPE'] = 0.88 / 1013 # RAMA5 - 
catch_data.loc[catch_data.CatchId == 932030072, 'SLOPE']=1.75 / 635  # RAMA5
catch_data.loc[catch_data.CatchId == 6250680, 'SLOPE'] = 0.00005     # ROCK2 - arbitrary value to remove error

# 020402
catch_data.loc[catch_data.CatchId == 4491888, 'SLOPE'] = 0.5 / 3571 # NBRAN4

catch_data.loc[catch_data.SLOPE < 0.00003, 'SLOPE'] = 0.00003

In [4]:
file = './HWM/Iter4_USGS/USGS_Opt.csv'

# print(file[:30])
rough_data = pd.read_csv(file)
rough_data = rough_data.drop_duplicates(subset='COMID')
rough_data = rough_data.dropna(subset=['COMID'])
rough_data

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,agency_cd,site_no,datetime,tz_cd,194264_00065,Stream_H_m,194264_00065_cd,OID_,Field1,REACHCODE,REACHSMDAT,REACHRESOL,FEATURECOM,FEATURECLA,SOURCE_ORI,SOURCE_DAT,SOURCE_FEA,FEATUREDET,Measure,Offset,EventType,FLComID,POINT_X,POINT_Y,POINT_Z,POINT_M,COMID,time_file,Predict_Stream_H_m,roughness,rmse
0,0,0,0,0,USGS,1389802,2011-08-30 03:00,EDT,122.54,37.350192,A,708,2203,2.030000e+12,NaN,Medium,0,0,"USGS, Water Resources Division",NaN,1389802,http://waterdata.usgs.gov/nwis/nwisman/?site_n...,79.85481,0,StreamGage,6251048,-74.181557,40.915657,0,79.85481,6251048,20110829_01,7.125234,0.200000,3.022496e+01
1,1,1,1,1,USGS,1403060,2011-08-28 18:00,EDT,41.90,12.771120,A,490,1865,2.030000e+12,NaN,Medium,0,0,"USGS, Water Resources Division",NaN,1403060,http://waterdata.usgs.gov/nwis/nwisman/?site_n...,68.90423,0,StreamGage,9513378,-74.548574,40.551601,0,68.90423,9513378,20110828_19,12.771120,0.188120,2.587185e-07
2,2,2,2,2,USGS,1400500,2011-08-28 16:00,EDT,26.29,8.013192,A,491,2015,2.030000e+12,NaN,Medium,0,0,"USGS, Water Resources Division",NaN,1400500,http://waterdata.usgs.gov/nwis/nwisman/?site_n...,89.83748,0,StreamGage,9513852,-74.583119,40.555555,0,89.83748,9513852,20110828_18,8.013193,0.010610,1.113257e-06
3,3,3,3,3,USGS,1388500,2011-08-29 01:15,EDT,25.24,7.693152,A,720,2466,2.030000e+12,NaN,Medium,0,0,"USGS, Water Resources Division",NaN,1388500,http://waterdata.usgs.gov/nwis/nwisman/?site_n...,56.37115,0,StreamGage,6249470,-74.281760,40.969298,0,56.37115,6249470,20110828_16,7.693153,0.031575,5.453923e-07
4,4,4,4,4,USGS,1446995,2011-08-29 05:00,EDT,25.15,7.665720,A,565,2389,2.040000e+12,NaN,Medium,0,0,"USGS, Water Resources Division",NaN,1446995,http://waterdata.usgs.gov/nwis/nwisman/?site_n...,86.53518,0,StreamGage,2588457,-75.203571,40.694496,0,86.53518,2588457,20110830_19,7.665722,0.026794,1.504787e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,128,129,147,149,USGS,1409000,2011-08-28 09:45,EDT,4.32,1.316736,A,255,1940,2.040000e+12,NaN,Medium,0,0,"USGS, Water Resources Division",NaN,1409000,http://waterdata.usgs.gov/nwis/nwisman/?site_n...,18.62468,0,StreamGage,9453855,-74.169190,39.867962,0,18.62468,9453855,20110828_08,1.316736,0.061568,6.343089e-08
129,129,130,148,150,USGS,1396800,2011-08-28 14:30,EDT,4.06,1.237488,A,569,1981,2.030000e+12,NaN,Medium,0,0,"USGS, Water Resources Division",NaN,1396800,http://waterdata.usgs.gov/nwis/nwisman/?site_n...,0.00322,0,StreamGage,9513994,-74.915253,40.639259,0,0.00322,9512994,20110828_09,2.297540,0.005000,1.060052e+00
130,130,131,149,151,USGS,1379773,2011-08-28 19:45,EDT,3.84,1.170432,A,692,2638,2.030000e+12,NaN,Medium,0,0,"USGS, Water Resources Division",NaN,1379773,http://waterdata.usgs.gov/nwis/nwisman/?site_n...,13.19529,0,StreamGage,6249510,-74.539606,40.959445,0,13.19529,6249510,20110828_13,1.170432,0.170180,2.234323e-08
131,131,132,150,153,USGS,1443900,2011-08-28 10:30,EDT,3.22,0.981456,A,664,2567,2.040000e+12,NaN,Medium,0,0,"USGS, Water Resources Division",NaN,1443900,http://waterdata.usgs.gov/nwis/nwisman/?site_n...,62.34066,0,StreamGage,2587723,-75.039398,40.981147,0,62.34066,2587723,20110828_10,0.981456,0.098805,3.784705e-08


In [10]:
data = catch_data.merge(rough_data[['COMID','roughness']], left_on='CatchId', right_on='COMID')
data = data.drop(columns=['CatchId','COMID','Unnamed: 0','Roughness'])
data

,SLOPE,LENGTHKM,AREASQKM,StreamOrde,Avg_Lon,Avg_Lat,Avg_Elev,RunoffC,MajLC,roughness
0,0.022008,0.961,0.6426,6.0,-74.183398,40.912749,49.183880,70.19608,23.0,0.200000
1,0.000030,0.714,0.8109,6.0,-74.230063,40.880505,59.897620,65.97669,22.0,0.005000
2,0.000656,5.596,21.5631,6.0,-74.138098,40.899507,27.244300,73.41438,23.0,0.005000
3,0.000030,1.267,1.7118,5.0,-74.313445,40.865949,52.020800,75.44427,23.0,0.160339
4,0.003309,3.841,4.9347,4.0,-74.387943,40.899601,90.125530,52.77950,21.0,0.034440
...,...,...,...,...,...,...,...,...,...,...
126,0.000653,4.318,9.1620,4.0,-74.189969,39.868283,8.961771,63.08644,90.0,0.061568
127,0.000627,7.034,18.1917,4.0,-74.221220,39.984090,14.696730,61.71870,22.0,0.184081
128,0.000702,4.332,6.1263,4.0,-74.554826,39.693357,13.715910,54.70472,43.0,0.200000
129,0.001370,3.963,12.6657,3.0,-74.316724,39.657761,10.979880,55.16379,43.0,0.200000


In [7]:
data.describe()

,SLOPE,LENGTHKM,AREASQKM,StreamOrde,Avg_Lon,Avg_Lat,Avg_Elev,RunoffC,MajLC,Roughness,roughness
count,131.000000,131.000000,131.000000,131.000000,131.000000,131.000000,131.000000,131.000000,131.000000,131.000000,131.000000
mean,0.004047,2.706214,4.883496,3.343511,-74.578767,40.518392,72.880878,58.305164,44.053435,0.068150,0.116125
std,0.006406,2.542440,6.480313,1.363090,0.341714,0.460217,77.709547,9.061670,26.001572,0.058371,0.077576
min,0.000030,0.129000,0.029700,1.000000,-75.331314,39.303889,1.483143,40.573250,11.000000,0.005000,0.005000
25%,0.000541,0.957500,0.932850,2.000000,-74.826009,40.253784,19.792810,52.247975,22.000000,0.025944,0.035808
50%,0.001370,2.067000,2.858400,3.000000,-74.581197,40.618870,38.141300,58.862780,41.000000,0.055004,0.125701
75%,0.003774,3.831000,6.209550,4.000000,-74.304378,40.883764,92.341955,64.174695,43.000000,0.093453,0.200000
max,0.033953,18.190000,46.157400,6.000000,-73.725891,41.214850,327.668800,80.133330,90.000000,0.294773,0.200000


In [8]:
data.loc[data['StreamOrde'] == 6.0]

,SLOPE,LENGTHKM,AREASQKM,StreamOrde,Avg_Lon,Avg_Lat,Avg_Elev,RunoffC,MajLC,Roughness,roughness
0,0.022008,0.961,0.6426,6.0,-74.183398,40.912749,49.18388,70.19608,23.0,0.145657,0.200000
1,0.000030,0.714,0.8109,6.0,-74.230063,40.880505,59.89762,65.97669,22.0,0.005000,0.005000
2,0.000656,5.596,21.5631,6.0,-74.138098,40.899507,27.24430,73.41438,23.0,0.006667,0.005000
8,0.000265,0.694,0.7299,6.0,-74.268617,40.897015,52.98941,68.97657,21.0,0.005000,0.025847
18,0.001936,4.086,9.7803,6.0,-74.245048,40.887024,71.50792,64.62179,22.0,0.012016,0.057234
50,0.002349,0.298,0.0675,6.0,-74.550166,40.550904,8.68687,80.13333,90.0,0.042761,0.188120
81,0.000030,3.549,14.2263,6.0,-74.973820,40.401568,72.54365,46.92383,41.0,0.009164,0.005000
84,0.000030,2.559,4.1652,6.0,-75.068758,40.536058,60.75852,56.03284,41.0,0.005000,0.005241
87,0.000506,6.480,29.9709,6.0,-74.810979,40.240866,27.97365,59.14988,21.0,0.055004,0.022359
88,0.000101,2.877,3.8547,6.0,-74.859220,40.294406,33.03311,55.90474,41.0,0.005000,0.006670


In [9]:
data.dtypes

SLOPE         float64
LENGTHKM      float64
AREASQKM      float64
StreamOrde    float64
Avg_Lon       float64
Avg_Lat       float64
Avg_Elev      float64
RunoffC       float64
MajLC         float64
Roughness     float64
roughness     float64
dtype: object

In [11]:
# Where the roughness calibration results in 0.3 or 0.005, this is likely an error related to flows
# We drop these data
# data = data[data.roughness < 0.1999]
# data = data[data.roughness > 0.0051]

# Add a bias term  -  This didn't do anything so I took it out
# data = pd.concat([pd.Series(1, index=data.index, name='Bias'), data], axis=1)

# Convert data types into forms usable for regression
data.MajLC = data.MajLC.astype(str)
data.StreamOrde = data.StreamOrde.astype(str)

# data = pd.concat([pd.Series(1, index=data.index, name='Bias'), data], axis=1)

X = data.copy()
# X = X.drop(columns=['fmax','COMID','RMSE']) #, 'Avg_Lon','Avg_Lat'])   # all data but roughness is the input
# X = X.drop(columns=['fmax','COMID','RMSE','Predict_Stream_H_m','Stream_H_m','Majority_Land_Use'])
# X = X.drop(columns=['fmax','COMID','RMSE','Predict_Stream_H_m','Stream_H_m',]) # 'Avg_Lon','Avg_Lat'])

# X.to_csv(file[:25] + 'HWM_regression_merge.csv')

X = X.drop(columns='roughness')   # all data but roughness is the input
X = pd.get_dummies(data=X) # turns categorical data into a form that regression can interpret
X

,SLOPE,LENGTHKM,AREASQKM,Avg_Lon,Avg_Lat,Avg_Elev,RunoffC,StreamOrde_1.0,StreamOrde_2.0,StreamOrde_3.0,StreamOrde_4.0,StreamOrde_5.0,StreamOrde_6.0,MajLC_11.0,MajLC_21.0,MajLC_22.0,MajLC_23.0,MajLC_41.0,MajLC_42.0,MajLC_43.0,MajLC_81.0,MajLC_82.0,MajLC_90.0
0,0.022008,0.961,0.6426,-74.183398,40.912749,49.183880,70.19608,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
1,0.000030,0.714,0.8109,-74.230063,40.880505,59.897620,65.97669,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0
2,0.000656,5.596,21.5631,-74.138098,40.899507,27.244300,73.41438,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
3,0.000030,1.267,1.7118,-74.313445,40.865949,52.020800,75.44427,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
4,0.003309,3.841,4.9347,-74.387943,40.899601,90.125530,52.77950,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,0.000653,4.318,9.1620,-74.189969,39.868283,8.961771,63.08644,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
127,0.000627,7.034,18.1917,-74.221220,39.984090,14.696730,61.71870,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
128,0.000702,4.332,6.1263,-74.554826,39.693357,13.715910,54.70472,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
129,0.001370,3.963,12.6657,-74.316724,39.657761,10.979880,55.16379,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0


In [12]:
X.dtypes

SLOPE             float64
LENGTHKM          float64
AREASQKM          float64
Avg_Lon           float64
Avg_Lat           float64
Avg_Elev          float64
RunoffC           float64
StreamOrde_1.0      uint8
StreamOrde_2.0      uint8
StreamOrde_3.0      uint8
StreamOrde_4.0      uint8
StreamOrde_5.0      uint8
StreamOrde_6.0      uint8
MajLC_11.0          uint8
MajLC_21.0          uint8
MajLC_22.0          uint8
MajLC_23.0          uint8
MajLC_41.0          uint8
MajLC_42.0          uint8
MajLC_43.0          uint8
MajLC_81.0          uint8
MajLC_82.0          uint8
MajLC_90.0          uint8
dtype: object

In [24]:
# These are the available data columns
# Stream_H_m COMID StreamOrde fmax Predict_Stream_H_m roughness RMSE Majority_Land_Use \
# Avg_Lon Avg_Lat Avg_Elev RunoffC SLOPE LENGTHKM AREASQKM

y = data.roughness # only roughness is the target
# X

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) #, random_state=5

reg = LinearRegression().fit(X_train, y_train)
print('Train R² = ' + str(round(reg.score(X_train, y_train),6)))
print('Test  R² = ' + str(round(reg.score(X_test, y_test),6)))

reg = LinearRegression().fit(X, y)
print('All   R² = ' + str(round(reg.score(X, y),6)))
x=0
for i in reg.coef_:
    print(f'{round(i,6)} * {X.columns[x]}')
#     if x == 0: print(" (Bias) ",end="")
#     if x > 0:  print(" * " + X.columns[x] + " ",end="")
#     print(" + ")
    x += 1

Train R² = 0.441138
Test  R² = 0.316831
All   R² = 0.453239
3.965064 * SLOPE
0.004948 * LENGTHKM
-0.000235 * AREASQKM
0.015848 * Avg_Lon
-0.011398 * Avg_Lat
4.5e-05 * Avg_Elev
0.002258 * RunoffC
0.017489 * StreamOrde_1.0
0.058344 * StreamOrde_2.0
0.019937 * StreamOrde_3.0
-0.010391 * StreamOrde_4.0
-0.033502 * StreamOrde_5.0
-0.051878 * StreamOrde_6.0
-0.065023 * MajLC_11.0
-0.018297 * MajLC_21.0
-0.016924 * MajLC_22.0
-0.045001 * MajLC_23.0
-0.014377 * MajLC_41.0
0.017291 * MajLC_42.0
0.085243 * MajLC_43.0
0.018274 * MajLC_81.0
0.016915 * MajLC_82.0
0.021898 * MajLC_90.0


In [25]:
# This was an experiment in normalizing the data. It yeilded the exact same results
X2 = X.apply(lambda x: (x-x.min()) / (x.max()-x.min()) , axis=0)
X2

y2 = y/y.max()
reg2 = LinearRegression().fit(X2, y2)
print('All   R² = ' + str(round(reg2.score(X2, y2),6)))

x=0
for i in reg2.coef_:
    print(f'{round(i,6)} * {X2.columns[x]}')
#     if x == 0: print(" (Bias) ",end="")
#     if x > 0:  print(" * " + X.columns[x] + " ",end="")
#     print(" + ")
    x += 1

All   R² = 0.453239
0.672544 * SLOPE
0.446809 * LENGTHKM
-0.054195 * AREASQKM
0.127212 * Avg_Lon
-0.108907 * Avg_Lat
0.072878 * Avg_Elev
0.446707 * RunoffC
0.087447 * StreamOrde_1.0
0.291722 * StreamOrde_2.0
0.099685 * StreamOrde_3.0
-0.051953 * StreamOrde_4.0
-0.16751 * StreamOrde_5.0
-0.259391 * StreamOrde_6.0
-0.325113 * MajLC_11.0
-0.091486 * MajLC_21.0
-0.084619 * MajLC_22.0
-0.225005 * MajLC_23.0
-0.071885 * MajLC_41.0
0.086457 * MajLC_42.0
0.426213 * MajLC_43.0
0.091372 * MajLC_81.0
0.084577 * MajLC_82.0
0.109488 * MajLC_90.0


3.965064 * SLOPE
0.004948 * LENGTHKM
-0.000235 * AREASQKM
0.015848 * Avg_Lon
-0.011398 * Avg_Lat
4.5e-05 * Avg_Elev
0.002258 * RunoffC
0.017489 * StreamOrde_1.0
0.058344 * StreamOrde_2.0
0.019937 * StreamOrde_3.0
-0.010391 * StreamOrde_4.0
-0.033502 * StreamOrde_5.0
-0.051878 * StreamOrde_6.0
-0.065023 * MajLC_11.0
-0.018297 * MajLC_21.0
-0.016924 * MajLC_22.0
-0.045001 * MajLC_23.0
-0.014377 * MajLC_41.0
0.017291 * MajLC_42.0
0.085243 * MajLC_43.0
0.018274 * MajLC_81.0
0.016915 * MajLC_82.0
0.021898 * MajLC_90.0


In [22]:
data

,SLOPE,LENGTHKM,AREASQKM,StreamOrde,Avg_Lon,Avg_Lat,Avg_Elev,RunoffC,MajLC,roughness
0,0.022008,0.961,0.6426,6.0,-74.183398,40.912749,49.183880,70.19608,23.0,0.200000
1,0.000030,0.714,0.8109,6.0,-74.230063,40.880505,59.897620,65.97669,22.0,0.005000
2,0.000656,5.596,21.5631,6.0,-74.138098,40.899507,27.244300,73.41438,23.0,0.005000
3,0.000030,1.267,1.7118,5.0,-74.313445,40.865949,52.020800,75.44427,23.0,0.160339
4,0.003309,3.841,4.9347,4.0,-74.387943,40.899601,90.125530,52.77950,21.0,0.034440
...,...,...,...,...,...,...,...,...,...,...
126,0.000653,4.318,9.1620,4.0,-74.189969,39.868283,8.961771,63.08644,90.0,0.061568
127,0.000627,7.034,18.1917,4.0,-74.221220,39.984090,14.696730,61.71870,22.0,0.184081
128,0.000702,4.332,6.1263,4.0,-74.554826,39.693357,13.715910,54.70472,43.0,0.200000
129,0.001370,3.963,12.6657,3.0,-74.316724,39.657761,10.979880,55.16379,43.0,0.200000


In [23]:
X

,SLOPE,LENGTHKM,AREASQKM,Avg_Lon,Avg_Lat,Avg_Elev,RunoffC,StreamOrde_1.0,StreamOrde_2.0,StreamOrde_3.0,StreamOrde_4.0,StreamOrde_5.0,StreamOrde_6.0,MajLC_11.0,MajLC_21.0,MajLC_22.0,MajLC_23.0,MajLC_41.0,MajLC_42.0,MajLC_43.0,MajLC_81.0,MajLC_82.0,MajLC_90.0
0,0.022008,0.961,0.6426,-74.183398,40.912749,49.183880,70.19608,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
1,0.000030,0.714,0.8109,-74.230063,40.880505,59.897620,65.97669,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0
2,0.000656,5.596,21.5631,-74.138098,40.899507,27.244300,73.41438,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
3,0.000030,1.267,1.7118,-74.313445,40.865949,52.020800,75.44427,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
4,0.003309,3.841,4.9347,-74.387943,40.899601,90.125530,52.77950,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,0.000653,4.318,9.1620,-74.189969,39.868283,8.961771,63.08644,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
127,0.000627,7.034,18.1917,-74.221220,39.984090,14.696730,61.71870,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
128,0.000702,4.332,6.1263,-74.554826,39.693357,13.715910,54.70472,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
129,0.001370,3.963,12.6657,-74.316724,39.657761,10.979880,55.16379,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0


In [19]:
huc = '020301'
path = 'C:/Users/David/G-Rutgers/' + huc + '/hydrogeo-fulltable-' + huc + '-updated-regr.csv'
hucdata_full = pd.read_csv(path) #, dtype={'Avg_Elev': int, 'Majority_Land_Use': int})

# Don't need the entire table, just one row for each COMID. Reorder to match regression
hucdata = hucdata_full.drop_duplicates(subset='CatchId')
hucdata = hucdata[['CatchId','Avg_Lon','Avg_Lat','Avg_Elev','RunoffC','SLOPE','LENGTHKM', 'AREASQKM', 'StreamOrde','Majority_Land_Use']]

# We are dropping the NaN data for simplicity at this time
hucdata = hucdata.fillna(-1)
hucdata = hucdata[hucdata != -1]
hucdata = hucdata.dropna()
hucdata = hucdata[hucdata.StreamOrde != -9]
# hucdata[hucdata.isna().any(axis=1)]
# hucdata[hucdata.isnull().any(axis=1)]

# Not all categorical data made it into the regression. Substitude unrepresented values with similars
hucdata.Majority_Land_Use[hucdata.Majority_Land_Use == 42] = 41
hucdata.Majority_Land_Use[hucdata.Majority_Land_Use == 43] = 41
hucdata.Majority_Land_Use[hucdata.Majority_Land_Use == 52] = 81
hucdata.Majority_Land_Use[hucdata.Majority_Land_Use == 71] = 81
hucdata.StreamOrde[hucdata.StreamOrde == 7] = 6

# Turn categories into strings. get_dummies turns categorical data into a form that regression can interpret
hucdata.StreamOrde = hucdata.StreamOrde.astype(str)
hucdata.Majority_Land_Use = hucdata.Majority_Land_Use.astype(str)
hucdata = pd.get_dummies(data=hucdata) 

# Create the input for the regression. Must match the columns of regression data
hucdata2 = hucdata.drop(columns=['CatchId',])
predict = reg.predict(hucdata2)

# Write predictions to dataframe. Fit to bounds
hucdata['Roughness2'] = ''
hucdata['Roughness2'] = predict
hucdata['Roughness2'][hucdata['Roughness2'] < 0.005] = 0.005
hucdata['Roughness2'][hucdata['Roughness2'] > 0.300] = 0.300

# Create a df with just catchment and roughness. Merge back on to original rating curve table.
merge = hucdata[['CatchId','Roughness2']]
hucdata_full = hucdata_full.merge(merge, on='CatchId',how='left')

# Fill roughness values where regression data was not available.
hucdata_full.Roughness2 = hucdata_full.Roughness2.fillna(0.1)

for catch in data.COMID.unique():
    subset = data[data['COMID'] == catch]
    # local_rough = opt['roughness'].loc[opt['COMID'] == catch][0]
    local_rough = subset['roughness'].values[0]
    hucdata_full['Roughness2'].loc[hucdata_full['CatchId'] == catch] = local_rough
    print('Rough at ' + str(catch) + ' is ' + str(local_rough))
    
# Calculate discharge with the new regression-determined roughess
hucdata_full['Discharge (m3s-1)'] = hucdata_full['WetArea (m2)'] * pow(hucdata_full['HydraulicRadius (m)'],2.0/3) * \
        pow(hucdata_full['SLOPE'],0.5) / hucdata_full['Roughness2']

# hucdata_full.to_csv(path[:-4] + '-calc.csv',index=False)

C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A valu

Rough at 4481733 is 0.039937296
Rough at 2590137 is 0.02969414
Rough at 2589827 is 0.032617394
Rough at 2590117 is 0.158865421
Rough at 9485472 is 0.026918167
Rough at 9481158 is 0.258248615
Rough at 4481599 is 0.00750001
Rough at 4481849 is 0.109510409
Rough at 4481249 is 0.197642486
Rough at 4480945 is 0.052991782
Rough at 4480899 is 0.01687911
Rough at 6251150 is 0.006422859
Rough at 9442271 is 0.09398781
Rough at 9441129 is 0.134103371
Rough at 9442071 is 0.174249212
Rough at 9441119 is 0.107788208
Rough at 9484602 is 0.084514293
Rough at 9481798 is 0.168884791
Rough at 9515628 is 0.070348176
Rough at 9515706 is 0.132828043
Rough at 9515652 is 0.099875061
Rough at 6249432 is 0.091387272
Rough at 9513008 is 0.058429496
Rough at 9512922 is 0.039589343
Rough at 9512876 is 0.053620068
Rough at 4491898 is 0.061226991
Rough at 4490670 is 0.04894122
Rough at 4492212 is 0.196415477
Rough at 6248502 is 0.013198912
Rough at 6249408 is 0.027666937
Rough at 6249372 is 0.054975505
Rough at 6246

C:\Users\David\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\David\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\David\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-

Rough at 6250618 is 0.045754166
Rough at 6249402 is 0.032993998
Rough at 6249352 is 0.025574786
Rough at 6246210 is 0.016934515
Rough at 6249816 is 0.01107363
Rough at 6249690 is 0.074573726
Rough at 4495848 is 0.058326364
Rough at 4495254 is 0.123373058
Rough at 4495824 is 0.091895989
Rough at 4495366 is 0.061370529
Rough at 4495376 is 0.063696226
Rough at 4496468 is 0.018840515
Rough at 6250952 is 0.00941271
Rough at 6246148 is 0.022491129
Rough at 6246132 is 0.007421885
Rough at 6246098 is 0.05875399
Rough at 6245796 is 0.041908057
Rough at 6245718 is 0.02729472
Rough at 6246322 is 0.035299784
Rough at 9514052 is 0.007045592
Rough at 9513352 is 0.115684076
Rough at 6250706 is 0.052675626
Rough at 6250654 is 0.048020155
Rough at 6249904 is 0.033424361
Rough at 6251208 is 0.076678359
Rough at 6250650 is 0.038035548
Rough at 6250680 is 0.00616767
Rough at 6249912 is 0.03952802
Rough at 6249860 is 0.035026819
Rough at 6250646 is 0.156015751
Rough at 6249724 is 0.00812501
Rough at 624977

C:\Users\David\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\David\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\David\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-

Rough at 6246064 is 0.010436593
Rough at 6250742 is 0.099801333
Rough at 6250738 is 0.053452431
Rough at 6250088 is 0.020092909
Rough at 6250748 is 0.089489265
Rough at 6250092 is 0.16644948
Rough at 6250744 is 0.157452244
Rough at 6250138 is 0.084926889
Rough at 9513054 is 0.033968304
Rough at 9514722 is 0.00604965
Rough at 9513572 is 0.022234814
Rough at 9513154 is 0.032807211
Rough at 9513310 is 0.075718993
Rough at 9515554 is 0.015843872
Rough at 9514754 is 0.151644891
Rough at 9513328 is 0.066407128
Rough at 9513340 is 0.139792583
Rough at 9513658 is 0.195901537
Rough at 9513872 is 0.055999111
Rough at 9513148 is 0.146782345
Rough at 9513840 is 0.083522113
Rough at 9517822 is 0.007851013
Rough at 9515544 is 0.012351846
Rough at 9514038 is 0.027722716
Rough at 9517826 is 0.065427781
Rough at 9514706 is 0.009896684
Rough at 9514870 is 0.011156009
Rough at 9514744 is 0.011833764
Rough at 9513382 is 0.00664403
Rough at 9513372 is 0.071098815
Rough at 9513452 is 0.014152841
Rough at 95

In [10]:
# hucs = ['020301','020401','020402','020403']
# for huc in hucs:
#     path = 'C:/Users/David/G-Rutgers/' + huc + '/hydrogeo-fulltable-' + huc + '-updated-regr.csv'
#     hucdata = pd.read_csv(path)
    
    